In [33]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize
from scipy.special import comb
from scipy.stats import norm

# Valeurs initiales

In [44]:
L = 10 #time horizon
nb_iter = 100 #nb d'itérations pour la simulation de Montecarlo
m = 30 #dimension de l'espace de projection

X_0 = 100 # prix initial
K_0 = 85 #strike
sigma = 0.2 # volatility

valeurs_sigma = np.linspace(0.05, 0.25, 10)
valeurs_strike = np.linspace(85, 120, 10)

In [45]:
def binom_EU1(S0, K , T, r, sigma, N, type_ = 'call'):
    dt = T/N
    u = np.exp(sigma * np.sqrt(dt))
    d = np.exp(-sigma * np.sqrt(dt))
    p = (  np.exp(r*dt) - d )  /  (  u - d )
    value = 0 
    for i in range(N+1):
        node_prob = comb(N, i)*p**i*(1-p)**(N-i)
        ST = S0*(u)**i*(d)**(N-i)
        if type_ == 'call':
            value += max(ST-K,0) * node_prob
        elif type_ == 'put':
            value += max(K-ST, 0)*node_prob
        else:
            raise ValueError("type_ must be 'call' or 'put'" )
    
    return value*np.exp(-r*T)

In [46]:
binom_EU1(X_0, K_0, L, 0, sigma, L, type_='call')

31.646099631307656

In [47]:
binomial_premium1 = []
for strike in valeurs_strike:
    binomial_premium1.append(binom_EU1(X_0, strike, L, 0, sigma, L, type_='put'))
    
'''binomial_premium2 = []
for sig in valeurs_sigma:
    binomial_premium2.append(binom_EU1(X_0, K_0, L, 0, sig, L, type_='put'))
'''

"binomial_premium2 = []\nfor sig in valeurs_sigma:\n    binomial_premium2.append(binom_EU1(X_0, K_0, L, 0, sig, L, type_='put'))\n"

In [48]:
print(binomial_premium1)

[16.646099631307663, 18.60599436549075, 20.56588909967384, 22.525783833856934, 24.61574027513224, 27.486066958960798, 30.356393642789357, 33.22672032661793, 36.09704701044649, 38.96737369427505]


In [49]:
def black_scholes(S, K, T, r, sigma, option_type='call'):
    """
    Calculates the Black-Scholes option price for a European call or put option.
    
    Parameters:
    S (float): Spot price of the underlying asset.
    K (float): Strike price of the option.
    T (float): Time to maturity (in years).
    r (float): Risk-free interest rate (annual).
    sigma (float): Volatility of the underlying asset (annual).
    option_type (str): Type of the option ('call' or 'put').
    
    Returns:
    float: Price of the European option (call or put).
    """
    # Calculate d1 and d2 components of the Black-Scholes formula
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        # Calculate the call option price
        option_price = (S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2))
    elif option_type == 'put':
        # Calculate the put option price
        option_price = (K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1))
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    
    return option_price

In [50]:
black_scholes(X_0, valeurs_strike, L, 0, sigma, option_type='put')

array([16.18272117, 18.27413591, 20.47197104, 22.77064638, 25.16467397,
       27.64870544, 30.21756639, 32.8662806 , 35.59008578, 38.38444297])

In [51]:
black_scholes(X_0, K_0, L, 0, valeurs_sigma, option_type='put')

array([ 1.14914138,  2.9389606 ,  5.05103614,  7.31213907,  9.64631705,
       12.01518293, 14.39676433, 16.77711271, 19.14658236, 21.49802233])